In [13]:
"""
Pyroshell Market Sizing — Figures Pack (US + Canada)

REQUIRED (kept):
1) US  TAM/SAM/SOM (assets)
2) CA  TAM/SAM/SOM (assets)
3) Combined (assets) — stacked US + CA for TAM, SAM, SOM
4) Combined (one-time $) — stacked US + CA for TAM, SAM, SOM

ADDED (useful extras, consistent colors):
5) Compare US vs CA — TAM (assets)
6) Compare US vs CA — SAM (assets)
7) Compare US vs CA — SOM (assets)
8) Compare US vs CA — TAM (one-time $)
9) Compare US vs CA — SAM (one-time $)
10) Compare US vs CA — SOM (one-time $)
11) Sensitivity — US TAM (one-time $) vs Exposure % (25–40)
12) Sensitivity — SOM (3-yr assets) vs Capacity (installs/week per crew)

Outputs to: ./image
"""

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# =========================
# PARAMETERS (edit as needed)
# =========================
# Unit counts
US_SITES = 450_000
CA_SITES = 56_700

# Exposure assumptions
US_EXPOSURE = 0.33
CA_EXPOSURE_BASE = 0.20   # base (you may add low/high in your doc text)

# Price & service
ASP = 4_500               # $/site installed
SERVICE_ATTACH = 0.40
SERVICE_FEE = 350

# Phase-1 footprint (SAM) & Gen-1 fit
US_WEST_SHARE = 0.55
CA_WEST_SHARE = 0.50
GEN1_FIT = 0.80

# SOM capacity & ramp (shared NA crew pool split 80/20)
NA_CREWS = 6
INSTALLS_PER_WEEK = 4
WEEKS_PER_YEAR = 45
ADOPTION_RAMP = [0.50, 0.85, 1.00]
US_CAPACITY_SHARE = 0.80
CA_CAPACITY_SHARE = 0.20

# Output folder (relative)
OUT_DIR = Path("image")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# CONSISTENT COLORS (Okabe–Ito palette)
# =========================
C_US = "#0072B2"      # blue (United States)
C_CA = "#009E73"      # green (Canada)
C_TAM = "#A6CEE3"     # light blue (tier)
C_SAM = "#1F78B4"     # medium blue (tier)
C_SOM = "#0B4F79"     # dark blue (tier)
EDGE = "#333333"

C_US = "#0A3161"      # blue for United States
C_CA = "#EF3340"      # red for Canada
EDGE = "#333333"      # outline for readability
C_US_light = "#2F5F9E"
C_CA_light = "#F36A73"


# =========================
# HELPERS
# =========================
def som_over_3yrs(sam_assets: float, cap_per_year: float, ramp):
    installs = []
    remaining = float(sam_assets)
    for r in ramp:
        x = min(r * cap_per_year, max(remaining, 0))
        installs.append(x)
        remaining -= x
    return sum(installs), installs

def add_value_labels(ax, xs, ys, fmt):
    for x, y in zip(xs, ys):
        ax.text(x, y, fmt(y), ha="center", va="bottom")

def fmt_int(v):      # whole numbers with commas
    return f"{int(round(v)):,}"

def fmt_money(v):    # dollars in millions
    return f"${v/1e6:.1f}M"

def bar_simple(labels, values, colors, title, ylabel, out_name, value_fmt):
    fig, ax = plt.subplots(figsize=(7.5, 5))
    xs = np.arange(len(labels))
    ax.bar(labels, values, color=colors, edgecolor=EDGE)
    add_value_labels(ax, xs, values, value_fmt)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.tight_layout()
    p = OUT_DIR / out_name
    plt.savefig(p, dpi=300)
    plt.close()
    print(f"[Saved] {p.resolve()}")

# =========================
# CORE CALCULATIONS
# =========================
# TAM assets
US_TAM_assets = US_SITES * US_EXPOSURE
CA_TAM_assets = CA_SITES * CA_EXPOSURE_BASE

# SAM assets
US_SAM_assets = US_TAM_assets * US_WEST_SHARE * GEN1_FIT
CA_SAM_assets = CA_TAM_assets * CA_WEST_SHARE * GEN1_FIT

# SOM capacity & installs
capacity_total = NA_CREWS * INSTALLS_PER_WEEK * WEEKS_PER_YEAR
US_capacity = capacity_total * US_CAPACITY_SHARE
CA_capacity = capacity_total * CA_CAPACITY_SHARE

US_SOM_assets, US_installs_by_year = som_over_3yrs(US_SAM_assets, US_capacity, ADOPTION_RAMP)
CA_SOM_assets, CA_installs_by_year = som_over_3yrs(CA_SAM_assets, CA_capacity, ADOPTION_RAMP)

# One-time revenue (ASP per site)
US_TAM_rev = US_TAM_assets * ASP
CA_TAM_rev = CA_TAM_assets * ASP
US_SAM_rev = US_SAM_assets * ASP
CA_SAM_rev = CA_SAM_assets * ASP
US_SOM_rev = US_SOM_assets * ASP
CA_SOM_rev = CA_SOM_assets * ASP

# Combined (for stacked charts)
COMB_TAM_assets = US_TAM_assets + CA_TAM_assets
COMB_SAM_assets = US_SAM_assets + CA_SAM_assets
COMB_SOM_assets = US_SOM_assets + CA_SOM_assets
COMB_TAM_rev = US_TAM_rev + CA_TAM_rev
COMB_SAM_rev = US_SAM_rev + CA_SAM_rev
COMB_SOM_rev = US_SOM_rev + CA_SOM_rev

# =========================
# 1) US — TAM/SAM/SOM (assets)
# =========================
bar_simple(
    labels=["TAM", "SAM", "SOM"],
    values=[US_TAM_assets, US_SAM_assets, US_SOM_assets],
    colors=[C_TAM, C_SAM, C_SOM],
    title="United States — TAM / SAM / SOM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_US_TAM_SAM_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# 2) Canada — TAM/SAM/SOM (assets)
# =========================
bar_simple(
    labels=["TAM", "SAM", "SOM"],
    values=[CA_TAM_assets, CA_SAM_assets, CA_SOM_assets],
    colors=[C_TAM, C_SAM, C_SOM],
    title="Canada (Base) — TAM / SAM / SOM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_CA_TAM_SAM_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# 3) Combined (assets), stacked US + Canada for each tier
# =========================
fig, ax = plt.subplots(figsize=(7.8, 5.2))
x = np.arange(3)  # TAM, SAM, SOM
us_vals = np.array([US_TAM_assets, US_SAM_assets, US_SOM_assets], dtype=float)
ca_vals = np.array([CA_TAM_assets, CA_SAM_assets, CA_SOM_assets], dtype=float)

ax.bar(x, us_vals, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_vals, bottom=us_vals, color=C_CA, edgecolor=EDGE, label="Canada (base)")
add_value_labels(ax, x, us_vals + ca_vals, fmt_int)

ax.set_xticks(x)
ax.set_xticklabels(["TAM", "SAM", "SOM"])
ax.set_ylabel("Number of assets (sites)")
ax.set_title("Assets Summary")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_combined_TAM_SAM_SOM_assets_stacked.png"
plt.savefig(p, dpi=300)
plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# 4) Combined (one-time $), stacked US + Canada for each tier
# =========================
fig, ax = plt.subplots(figsize=(7.8, 5.2))
us_rev = np.array([US_TAM_rev, US_SAM_rev, US_SOM_rev], dtype=float)
ca_rev = np.array([CA_TAM_rev, CA_SAM_rev, CA_SOM_rev], dtype=float)

ax.bar(x, us_rev, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_rev, bottom=us_rev, color=C_CA, edgecolor=EDGE, label="Canada (base)")
add_value_labels(ax, x, us_rev + ca_rev, fmt_money)

ax.set_xticks(x)
ax.set_xticklabels(["TAM", "SAM", "SOM"])
ax.set_ylabel("One-time revenue (USD)")
ax.set_title("One-time Revenue Summary $")
ax.legend()
plt.tight_layout()
p = OUT_DIR / "fig_combined_TAM_SAM_SOM_revenue_stacked.png"
plt.savefig(p, dpi=300)
plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# ADDED — US vs Canada comparisons (assets)
# =========================
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_TAM_assets, CA_TAM_assets],
    colors=[C_US, C_CA],
    title="TAM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_TAM_assets.png",
    value_fmt=fmt_int
)

bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SAM_assets, CA_SAM_assets],
    colors=[C_US, C_CA],
    title="SAM (Assets)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_SAM_assets.png",
    value_fmt=fmt_int
)

bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SOM_assets, CA_SOM_assets],
    colors=[C_US, C_CA],
    title="SOM (Assets, 3-yr)",
    ylabel="Number of assets (sites)",
    out_name="fig_compare_US_vs_CA_SOM_assets.png",
    value_fmt=fmt_int
)

# =========================
# ADDED — US vs Canada comparisons (one-time $)
# =========================
bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_TAM_rev, CA_TAM_rev],
    colors=[C_US, C_CA],
    title="TAM (One-time $)",
    ylabel="USD",
    out_name="fig_compare_US_vs_CA_TAM_revenue.png",
    value_fmt=fmt_money
)

bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SAM_rev, CA_SAM_rev],
    colors=[C_US, C_CA],
    title="SAM (One-time $)",
    ylabel="USD",
    out_name="fig_compare_US_vs_CA_SAM_revenue.png",
    value_fmt=fmt_money
)

bar_simple(
    labels=["United States", "Canada (base)"],
    values=[US_SOM_rev, CA_SOM_rev],
    colors=[C_US, C_CA],
    title="SOM (One-time $, 3-yr)",
    ylabel="USD",
    out_name="fig_compare_US_vs_CA_SOM_revenue.png",
    value_fmt=fmt_money
)

# =========================
# ADDED — Sensitivity: US TAM (one-time $) vs Exposure %
# =========================
exp_range = np.linspace(0.25, 0.40, 16)  # 25% to 40%
us_tam_rev_sens = US_SITES * exp_range * ASP

fig, ax = plt.subplots(figsize=(7.5, 5))
ax.plot(exp_range * 100, us_tam_rev_sens / 1e6, color=C_US)
ax.set_xlabel("U.S. exposure (%)")
ax.set_ylabel("U.S. TAM one-time (USD, millions)")
ax.set_title("Sensitivity — U.S. TAM (One-time $) vs Exposure %")
ax.grid(True, alpha=0.2)
plt.tight_layout()
p = OUT_DIR / "fig_sensitivity_US_TAM_vs_exposure.png"
plt.savefig(p, dpi=300)
plt.close()
print(f"[Saved] {p.resolve()}")

# =========================
# ADDED — Sensitivity: SOM (3-yr assets) vs installs/week per crew
# =========================
crew_week_range = np.arange(2, 8+1)  # from 2 to 8 installs/week/crew
som_totals = []
for iw in crew_week_range:
    cap_total = NA_CREWS * iw * WEEKS_PER_YEAR
    us_cap = cap_total * US_CAPACITY_SHARE
    ca_cap = cap_total * CA_CAPACITY_SHARE
    us_som, _ = som_over_3yrs(US_SAM_assets, us_cap, ADOPTION_RAMP)
    ca_som, _ = som_over_3yrs(CA_SAM_assets, ca_cap, ADOPTION_RAMP)
    som_totals.append(us_som + ca_som)

fig, ax = plt.subplots(figsize=(7.5, 5))
ax.plot(crew_week_range, som_totals, marker="o", color=C_SOM)
ax.set_xlabel("Installs per week per crew")
ax.set_ylabel("SOM (3-yr installs, assets)")
ax.set_title("Sensitivity — SOM (3-yr assets) vs Crew Throughput")
ax.grid(True, alpha=0.2)
plt.tight_layout()
p = OUT_DIR / "fig_sensitivity_SOM_vs_installs_per_week.png"
plt.savefig(p, dpi=300)
plt.close()
print(f"[Saved] {p.resolve()}")

# --- Optional console recap ---
print("\n--- Recap ---")
print(f"US  TAM/SAM/SOM assets: {fmt_int(US_TAM_assets)} / {fmt_int(US_SAM_assets)} / {fmt_int(US_SOM_assets)}")
print(f"CA  TAM/SAM/SOM assets: {fmt_int(CA_TAM_assets)} / {fmt_int(CA_SAM_assets)} / {fmt_int(CA_SOM_assets)}")
print(f"US  $ TAM/SAM/SOM: {fmt_money(US_TAM_rev)} / {fmt_money(US_SAM_rev)} / {fmt_money(US_SOM_rev)}")
print(f"CA  $ TAM/SAM/SOM: {fmt_money(CA_TAM_rev)} / {fmt_money(CA_SAM_rev)} / {fmt_money(CA_SOM_rev)}")


[Saved] D:\Repository\mech_431\image\fig_US_TAM_SAM_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_CA_TAM_SAM_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_combined_TAM_SAM_SOM_assets_stacked.png
[Saved] D:\Repository\mech_431\image\fig_combined_TAM_SAM_SOM_revenue_stacked.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_TAM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SAM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SOM_assets.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_TAM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SAM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_compare_US_vs_CA_SOM_revenue.png
[Saved] D:\Repository\mech_431\image\fig_sensitivity_US_TAM_vs_exposure.png
[Saved] D:\Repository\mech_431\image\fig_sensitivity_SOM_vs_installs_per_week.png

--- Recap ---
US  TAM/SAM/SOM assets: 148,500 / 65,340 / 2,030
CA  TAM/SAM/SOM assets: 11,340 / 4,536 /

In [15]:
fig, ax = plt.subplots(figsize=(7.8, 5.2)); labels = ["TAM","SAM","SOM"]; x = np.arange(len(labels)); ax.bar(labels, us_vals, color=C_US, edgecolor=EDGE, label="United States"); ax.bar(labels, ca_vals, bottom=us_vals, color=C_CA, edgecolor=EDGE, label="Canada (base)"); totals = us_vals + ca_vals; [ax.text(i, t, f"{int(t):,}", ha="center", va="bottom") for i, t in enumerate(totals)]; ax.set_ylabel("Number of assets (sites)"); ax.set_title("TAM / SAM / SOM (Assets, Stacked)"); ax.legend(); plt.tight_layout(); 
plt.savefig(OUT_DIR / "fig_combined_TAM_SAM_SOM_assets_stacked_quick.png", dpi=300); 
plt.close()


In [16]:
# --- New plot: Subscription ARR ramp (Y1–Y3), stacked US + Canada ---
# Uses: US_installs_by_year, CA_installs_by_year, SERVICE_ATTACH, SERVICE_FEE, OUT_DIR, C_US, C_CA, EDGE
import numpy as np
import matplotlib.pyplot as plt

# Cumulative installed base by year
us_cum = np.cumsum([int(x) for x in US_installs_by_year])
ca_cum = np.cumsum([int(x) for x in CA_installs_by_year])

# Annual Recurring Revenue (ARR) per year from the installed base
us_arr = us_cum * SERVICE_ATTACH * SERVICE_FEE
ca_arr = ca_cum * SERVICE_ATTACH * SERVICE_FEE
years = ["Y1", "Y2", "Y3"]
x = np.arange(len(years))

fig, ax = plt.subplots(figsize=(7.8, 5.2))
ax.bar(x, us_arr, color=C_US, edgecolor=EDGE, label="United States")
ax.bar(x, ca_arr, bottom=us_arr, color=C_CA, edgecolor=EDGE, label="Canada (base)")

# Top labels (totals) in $M for readability
totals = us_arr + ca_arr
for i, t in enumerate(totals):
    ax.text(i, t, f"${t/1e6:.2f}M", ha="center", va="bottom")

ax.set_xticks(x)
ax.set_xticklabels(years)
ax.set_ylabel("Annual Recurring Revenue (USD)")
ax.set_title("Subscription ")
ax.legend()
plt.tight_layout()
plt.savefig(OUT_DIR / "fig_subscription_ARR_ramp_stacked.png", dpi=300)
plt.close()
